## Best Model 제작을 위한 실험 프로세스 

어떤 실험이 가능한가? 
- 학습 실험 과정 
    - 데이터를 변경하면서 모델 성능 확인 
    - Step 의 코드를 업데이트 하면서 학습 결과 확인
    - Step 의 파라미터를 변경하면서 모델 성능 확인
    - 특정 Step 만 집중적으로 재학습 
</br></br>
 
- 추론 실험 과정    
    - 이전 학습 실험의 모델로 실험 가능  
    - 고정된 모델로 데이터를 변경하면서 추론 결과 확인
    - 고정된 모델로 파라미터를 변경하면서 추론 결과 확인     

</br>

**추론 결과가 좋은 모델을 찾는 것을 목표로 한다. !!**

### Step0. 사전 준비물: 

##### 준비물1: 
```bash
    conda activate {ENV-NAME}           ## python main.py 실행 용 가상환경
    pip install ipykernel        
    python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}
```

##### 준비물2: 
Asset 에 self.asset.save_summary() (ALO API) 를 사용하여 학습 및 추론 결과를 저장합니다.    
- train/inference_pipeline 에서 선언되는 경우 에 따라 train/inference history 에 따로 저장

===========  
```python 
# {solution_name}/assets/output/asset_output.py
summary = {}
summary['result'] = # model.predict() # 'OK'                                            ## Mandatory
summary['score'] = # model.predict_proba() # 0.98                                       ## Mandatory
summary['note'] = # Score는 모델의 추론 예측 결과에 대한 확률 값을 나타냅니다.            ## Mandatory
summary['probability'] = # model.predict_proba() # {'OK': 0.65, 'NG1':0.25, 'NG2':0.1}  ## Optional

self.asset.save_summary(result=summary['result'], score=summary['score'], note=summary['note'], probability=summary['probability'])
```

In [4]:
## history table 을 시각적 표현을 위해 
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
from src.alo import ALO
import os 
from pprint import pprint
import pandas as pd

alo = ALO()
train_pipeline = alo.pipeline(pipes='train_pipeline')
inference_pipeline = alo.pipeline(pipes='inference_pipeline')

[INFO][PROCESS][2024-03-08 13:37:54,531]: alolib already exists in local path.
[INFO][PROCESS][2024-03-08 13:37:55,743]: Success installing alolib requirements.txt
[INFO][PROCESS][2024-03-08 13:37:55,747]: Requirement already satisfied: pyyaml==6.0.1 in /home/sehyun.song/miniforge3/envs/tcr/lib/python3.10/site-packages (from -r /home/sehyun.song/Project/alo/dev-240228/alolib/requirements.txt (line 1)) (6.0.1)

[INFO][PROCESS][2024-03-08 13:37:55,751]: Successfully loaded experimental plan yaml: 
 /home/sehyun.song/Project/alo/dev-240228/solution/experimental_plan.yaml
[INFO][PROCESS][2024-03-08 13:37:55,754]: Successfully loaded << experimental_plan.yaml >> from: 
 /home/sehyun.song/Project/alo/dev-240228/solution/experimental_plan.yaml
[INFO][META][2024-03-08 13:37:55,776]: Loaded solution_metadata: 
None

[INFO][PROCESS][2024-03-08 13:37:55,779]: Process start-time: 240308_133755
[INFO][META][2024-03-08 13:37:55,781]: ALO version = develop
[INFO][PROCESS][2024-03-08 13:37:55,783]: ==

### Step 1. 학습 과정을 실행
- setup() : step 별 code 를 git repository 에서 clone 하고, python package 를 설치 합니다. 
- load() : external_data 를 ALO 환경으로 load 합니다. 
- run() : step 을 pipeline 순서대로 실행 합니다. 
- save() : 학습 및 추론 결과를 *_artifact 에 저장 합니다. 실험 내용을 history 에 backup 합니다. 


history 폴더에 아래와 같은 내용들이 저장 됩니다. 


```bash
├── history 
     ├── train
     │   ├── {UTC}-{random}-{contents_name}
     │        ├── experimental_plan.yaml
     │        ├── register_source    ## train docker container 준비물
     │            ├── assets
     │            ├── solution
     │            ├── src
     │            ├── alolib
     │        ├── models
     │        ├── score
     │        ├── report
     │        ├── output
     │        ├── log
     │            ├── experimental_history.json  ## 학습 데이터, 코드, 파라미터 변경 사항 기록
     │
     ├── inference
     │   ├── {UTC}-{random}-{contents_name}
     │        ├── experimental_plan.yaml
     │        ├── register_source    ## inference docker container 준비물
     │            ├── assets
     │            ├── solution
     │            ├── src
     │            ├── alolib
     │        ├── score
     │        ├── extra_output
     │        ├── output
     │        ├── log
     │            ├── experimental_history.json  ## 추론 데이터, 코드, 파라미터 변경 사항 기록
 ```

In [ ]:
train_pipeline.setup()
train_pipeline.load()
train_pipeline.run()
train_pipeline.save()

실험 결과를 확인 합니다.    
- "data_id"    : 실험에 사용된 데이터가 일부라도 변경될 경우 ID 변경. 동일 데이터로 실험 결과 비교 목적     
- "code_id"    : step 코드가 일부라도 변경됨을 감지(간단 프린트 문 추가 만으로도 ID 변경). 동일 코드로 실험 결과 비교에 집중    
- "param_id"   : 파라미터가 추가 되거나 값이 변경될 경우 ID 변경     

In [2]:
table_list = train_pipeline.history()
df = pd.DataFrame(table_list)
display(df.head(15))

""


In [3]:
table_list

[]

### Step 2. 데이터 변경하여 실험하기

In [ ]:
new_data = './solution/sample_data2/train_data/'  ## 실험에 필요한 데이터로 변경 !!

train_pipeline.load(data_path=new_data)
train_pipeline.run()
train_pipeline.save()

In [ ]:
df = train_pipeline.history()
display(df.head(15))

### Step 3. 코드 변경하여 실행하기

In [ ]:
train_pipeline.run()
train_pipeline.save()

In [ ]:
df = train_pipeline.history()
display(df.head(15))

### Step 4. 파라미터 변경하여 실행하기

In [ ]:
train_pipeline.get_parameter('train')['n_estimators'] = 150
train_pipeline.get_parameter('train')
train_pipeline.run()
train_pipeline.save()

### Step 4-2: 파라미터 변경 & 특정 Step 만 반복 실행하기 

In [ ]:
for value in [100, 110, 120, 130]:
    train_pipeline.get_parameter('train')['n_estimators'] = value
    train_pipeline.run(steps = ['train'])
    train_pipeline.save()

In [ ]:
df = train_pipeline.history()
display(df.head(15))

### Step 5: 추론 결과를 확인 (데이터, 코드, 파라미터 변경 가)

In [ ]:
inference_pipeline.setup()
inference_pipeline.load()
inference_pipeline.run()
inference_pipeline.save()

In [ ]:
df2 = inference_pipeline.history()
display(df2.head(15))

### 실험 결과 등록 (train_id, inference_id 입력)

In [ ]:
import getpass

username = input('Username: ')
password = getpass.getpass('Password: ')

print("Your ID : ", username)
print("Your PW : ", password.replace(password, '*' * len(password)))

In [ ]:
infra_setup = "./setting/example_infra_config/infra_config.localtest.yaml"
solution_info ={
    'inference_only': False, # True, False
    'solution_update': False,
    'solution_name': 'titanic-exp-test1',
    'solution_type': 'private',
    'contents_type': {
            'support_labeling': False,
            'inference_result_datatype': 'table', # 'image'
            'train_datatype': 'table', # 'image'
            'labeling_column_name': ''
    },
    'train_gpu': False, ## cpu, gpu
    'inference_gpu': False,
    "inference_arm": False,  # amd, arm  
}


history table 에서 ID 를 확인하고, 입력하여 솔루션을 등록 한다. 
- 학습 실험 과 추론 실험 을 혼합하여 솔루션 등록이 가능 
- train_id, inference_id 가 None 인 경우, 마지막 실험 결과를 등록 
    - history 의 experimental_plan.yaml 를 참조하여 1회 실행하고, 이를 솔루션 등록 함 
    - history 가 오염 될 수 있음을 고려 

In [ ]:
register = alo.register(
    infra_setup=infra_setup,
    solution_info=solution_info,
    train_id='',   ## history 에서 확인 한 Train ID 입력 하세요.!!
    inference_id='',  ## history 에서 확인 한 Inference ID 를 입력 하세요 !!
)

register.login(username, password)

In [ ]:
register.debugging = True  ## default: False (skip 항목: docker 생성, solution 등록)
register.run(username=username, password=password)

In [ ]:
register.run_train(
    status_period = 10, ## 몇 초 간격으로 학습 상태를 체크할 것인지 설정
    delete_instance = False,   ## 학습 테스트에 사용한 solution instance & stream 을 삭제할 것인지 설정 
    delete_solution= False,  ## solution 을 삭제할 것인지 설정
    )